#  ELISE

In [ ]:
import os
import time
import re
import random
import requests
from datetime import datetime
import pandas as pd
import zipfile
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import TimeoutException
from bs4 import BeautifulSoup
import json
import csv

##### CHẶN THÔNG BÁO #####

options = Options()
options.add_argument("--disable-infobars")
options.add_argument("--disable-notifications")
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option('useAutomationExtension', False)
prefs = {
    "credentials_enable_service": False, 
    "profile.password_manager_enabled": False, 
}
options.add_experimental_option("prefs", prefs)


web_url = "https://elise.vn/ao-den-co-phoi-luoi-dinh-day-hoa-fs2412024tswobk.html"
driver = webdriver.Chrome(options=options)
driver.maximize_window()
driver.get(web_url)


##### MÃ SẢN PHẨM #####
sku_element = driver.find_element(By.CSS_SELECTOR, "div.product.attribute.sku")
sku_value = sku_element.text
print(sku_value)
time.sleep(2)

##### TÊN SẢN PHẨM #####
name_element = driver.find_element(By.CSS_SELECTOR, "div.product-name-label h1")
name_value = name_element.text
print(name_value)
time.sleep(2)

##### MÔ TẢ #####
try:
    description_element = driver.find_element(By.CSS_SELECTOR, "div.value.std")
    description_value = description_element.text.replace("\n", " _ ")
    time.sleep(2)
except:
    description_value = None

print(description_value)



##### PHẦN TRĂM GIẢM GIÁ #####
try:
    sale_label = driver.find_element(By.CSS_SELECTOR, "span.product-label.sale-label span")
    sale_value = sale_label.text
except:
    sale_value = None
print(sale_value)

##### GIÁ #####
product_info_price = driver.find_element(By.CSS_SELECTOR,"div.product-info-price")
try:
    special_price = product_info_price.find_element(By.CSS_SELECTOR,"span.special-price")
    old_price = product_info_price.find_element(By.CSS_SELECTOR,"span.old-price")

    special_price_value = special_price.text
    old_price_value = old_price.text
except:
    old_price_value = product_info_price.text
    special_price_value  = None
print(old_price_value)
print(special_price_value)

##### KÍCH THƯỚC #####
wrapper_element = driver.find_element(By.CSS_SELECTOR, "div.product-options-wrapper")
size_elements = wrapper_element.find_elements(By.CSS_SELECTOR, "div.swatch-option.text")

size_texts = [size_element.text for size_element in size_elements]
sizes_value = "_".join(size_texts)
print(size_texts)
time.sleep(2)

##### HÌNH ẢNH #####
image_urls = []
image_elements = driver.find_elements(By.CSS_SELECTOR, "div.product.item-image.imgzoom img")

for image_element in image_elements:
    image_url = image_element.get_attribute('src')
    image_urls.append(image_url)
print(image_urls)


In [ ]:
import os
import time
import re
import random
import requests
from datetime import datetime
import pandas as pd
import zipfile
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import TimeoutException
from bs4 import BeautifulSoup
import json
import csv

##### PATH + LINK #####

base_save_dir = r'E:\INTERN\PORTFOLIO\AUTOMATION\SYSTEM\data'
web_url = "https://elise.vn/"

##### FUNCTION TẠO THƯ MỤC #####

def extract_website_name(url):
    """
    Lấy tên website từ URL, ví dụ: 'https://elise.vn/' -> 'elise'
    """
    match = re.search(r"https?://(?:www\.)?([^./]+)", url)
    return match.group(1) if match else "unknown"

start_time = datetime.now()
year = start_time.strftime("%Y")
month = start_time.strftime("%m")
day = start_time.strftime("%d")
hour_minute = start_time.strftime("%H_%M")

def initialize_directory_structure(base_save_dir, web_url):
    """
    Tạo cấu trúc thư mục cơ bản theo yêu cầu:
    website > năm > tháng > ngày > giờ_phút > list_view | detail_view | CSVs
    """
    website_name = extract_website_name(web_url)
    
    website_dir = os.path.join(base_save_dir, website_name)
    os.makedirs(website_dir, exist_ok=True)
    
    list_info_all_path = os.path.join(website_dir, "list_info_all.csv")
    if not os.path.exists(list_info_all_path):
        with open(list_info_all_path, 'w', encoding='utf-8', newline='') as f:
            writer = csv.writer(f)
            writer.writerow(["url"])  
    
    year_dir = os.path.join(website_dir, year)
    month_dir = os.path.join(year_dir, month)
    day_dir = os.path.join(month_dir, day)
    time_dir = os.path.join(day_dir, hour_minute)
    
    os.makedirs(year_dir, exist_ok=True)
    os.makedirs(month_dir, exist_ok=True)
    os.makedirs(day_dir, exist_ok=True)
    os.makedirs(time_dir, exist_ok=True)
    
    list_view_dir = os.path.join(time_dir, "list_view")
    detail_view_dir = os.path.join(time_dir, "detail_view")
    
    os.makedirs(list_view_dir, exist_ok=True)
    os.makedirs(detail_view_dir, exist_ok=True)
    
    list_attempt_path = os.path.join(time_dir, "list_attempt.csv")
    list_add_path = os.path.join(time_dir, "list_add.csv")
    list_done_path = os.path.join(time_dir, "list_done.csv")
    detail_attempt_path = os.path.join(time_dir, "detail_attempt.csv")
    
    for path in [list_attempt_path, list_add_path, list_done_path]:
        if not os.path.exists(path):
            with open(path, 'w', encoding='utf-8', newline='') as f:
                writer = csv.writer(f)
                writer.writerow(["url"]) 
    
    if not os.path.exists(detail_attempt_path):
        with open(detail_attempt_path, 'w', encoding='utf-8', newline='') as f:
            writer = csv.writer(f)
            writer.writerow(["sku", "name", "sale", "old_price", "special_price", "sizes", "description", "url", "images"])
    
    print(f"Initialized directory structure for {website_name}")
    
    return {
        'website_dir': website_dir,
        'time_dir': time_dir,
        'list_view_dir': list_view_dir,
        'detail_view_dir': detail_view_dir,
        'list_attempt_path': list_attempt_path,
        'list_add_path': list_add_path,
        'list_done_path': list_done_path,
        'detail_attempt_path': detail_attempt_path,
        'list_info_all_path': list_info_all_path
    }

def load_csv(file_path):
    """Hàm để đọc dữ liệu từ file CSV và trả về một set."""
    if not os.path.exists(file_path):
        return set()
    
    with open(file_path, mode='r', encoding='utf-8') as file:
        reader = csv.reader(file)
        next(reader, None)  
        return {row[0] for row in reader if row and row[0] != "url"}

def append_csv(file_path, data):
    """Hàm để lưu dữ liệu vào file CSV theo kiểu append."""
    file_exists = os.path.isfile(file_path) and os.path.getsize(file_path) > 0
    
    with open(file_path, "a", encoding="utf-8", newline='') as f:
        writer = csv.writer(f)
        if not file_exists:
            writer.writerow(["url"])
        for item in data:
            writer.writerow([item])

def update_product_links(paths, product_links):
    """
    Cập nhật các file CSV theo quy trình:
    - So sánh list_attempt với list_info_all để tìm ra các links mới (list_add)
    - Thêm các links mới vào list_info_all và list_add
    - Thêm tất cả các links vào list_attempt
    """
    list_info_all = load_csv(paths['list_info_all_path'])
    list_attempt = set(product_links) 
    
    list_add = list_attempt - list_info_all
    
    if list_add:
        append_csv(paths['list_info_all_path'], list_add)
        append_csv(paths['list_add_path'], list_add)
    
    append_csv(paths['list_attempt_path'], list_attempt)
    
    print(f"Updated CSV files:")
    print(f"- Added {len(list_add)} new links to list_add.csv")
    print(f"- Added {len(list_attempt)} links to list_attempt.csv")
    
    return list_add

def create_collection_directory(list_view_dir, collection_name, pagination_text):
    """
    Tạo thư mục collection_{pagination} trong list_view
    """
    collection_dir = os.path.join(list_view_dir, f"{collection_name}_{pagination_text}")
    images_dir = os.path.join(collection_dir, "images")
    
    os.makedirs(collection_dir, exist_ok=True)
    os.makedirs(images_dir, exist_ok=True)
    
    print(f"Created collection directory: {collection_dir}")
    
    return collection_dir, images_dir

def save_collection_product_data(folder_path, products, actions):
    """Lưu dữ liệu sản phẩm từ collection view và trả về danh sách các links."""
    image_folder = os.path.join(folder_path, "images")
    
    product_links = []
    
    for product in products:
        try:
            product_element = product.find_element(By.CSS_SELECTOR, "div.product-top a")
            product_href = product_element.get_attribute('href')
            product_links.append(product_href)
            
            try:
                image_name = product.find_element(By.CSS_SELECTOR, "div.product.details.product-item-details h5.product.name.product-item-name").text
                image_name = re.sub(r'[\\/*?:"<>|]', "", image_name.replace(" ", "_")) + ".jpg"
            except:
                image_name = f"product_{len(product_links)}.jpg"
            
            image_element = product.find_element(By.CSS_SELECTOR, "div.product-top img")
            image_url = image_element.get_attribute("src")
            
            if image_url:
                response = requests.get(image_url)
                if response.status_code == 200:
                    image_path = os.path.join(image_folder, image_name)
                    with open(image_path, "wb") as file:
                        file.write(response.content)
                    print(f"Saved Image: {image_path}")
        except Exception as e:
            print(f"Error processing collection product: {e}")
    
    json_path = os.path.join(folder_path, "data.json")
    with open(json_path, "w", encoding="utf-8") as json_file:
        json.dump(product_links, json_file, ensure_ascii=False, indent=4)
    print(f"Saved Collection JSON: {json_path}")
    
    return product_links

def crawl_collections():
    """
    Hàm thực hiện việc cào tất cả các links sản phẩm từ các bộ sưu tập
    """
    paths = initialize_directory_structure(base_save_dir, web_url)
    
    driver = webdriver.Chrome(options=options)
    driver.maximize_window()
    time.sleep(5)
    
    lookbook_url = f"{web_url.rstrip('/')}/lookbook"
    driver.get(lookbook_url)
    time.sleep(5)
    
    actions = ActionChains(driver)
    
    visited_collections = set()
    collection_page = 1 
    
    while True:    
        collections = driver.find_elements(By.CSS_SELECTOR, "li.item-lookbook")
        print(f"Found {len(collections)} collections on page {collection_page}")
        
        for idx, collection in enumerate(collections):
            if idx >= 2:
                break
            try:
                url_element = collection.find_element(By.CSS_SELECTOR, "div.item-image a")
                url_value = url_element.get_attribute('href')
                
                if url_value in visited_collections:
                    continue
                
                visited_collections.add(url_value)
                
                name_element = collection.find_element(By.CSS_SELECTOR, "div.item-content-info div.content h1")
                name_value = name_element.text
                clean_name = re.sub(r'[\\/*?:"<>|]', "", name_value.replace(" ", "_"))
                
                print(f"Processing collection ({idx+1}/{len(collections)}): {name_value} - {url_value}")
                
                driver.execute_script(f"window.open('{url_value}');")
                driver.switch_to.window(driver.window_handles[-1])
                time.sleep(2)
                
                collection_dir, _ = create_collection_directory(paths['list_view_dir'], clean_name, "1")
                
                save_html_source(collection_dir, driver)
                
                products = driver.find_elements(By.CSS_SELECTOR, "div.item div.item-content-wrapper")
                print(f"Found {len(products)} products in collection: {name_value}")
                
                product_links = save_collection_product_data(collection_dir, products, actions)
                
                update_product_links(paths, product_links)
                
                driver.close()
                driver.switch_to.window(driver.window_handles[0])
                time.sleep(1)
                
            except Exception as e:
                print(f"Error processing collection: {e}")
                if len(driver.window_handles) > 1:
                    driver.close()
                    driver.switch_to.window(driver.window_handles[0])
        
        try:
            see_more = driver.find_element(By.CSS_SELECTOR, "div.lookbook-actions span.button")
            driver.execute_script("arguments[0].scrollIntoView(true);", see_more)
            time.sleep(1)
            see_more.click()
            print("Clicked 'Xem thêm' button")
            time.sleep(3)  
            collection_page += 1
        except Exception as e:
            print(f"Không tìm thấy nút 'Xem thêm' hoặc đã hết collections: {e}")
            break
    
    print(f"Hoàn thành quá trình cào collections, đã cào {len(visited_collections)} bộ sưu tập")
    driver.quit()
    
    return paths

def create_detail_directory(detail_view_dir, sku, product_name):
    """
    Tạo thư mục trong detail_view theo cấu trúc:
    part_X > {năm_tháng_ngày_giờ_phút}_{sku}_{name}
    """
    part_folders = [f for f in os.listdir(detail_view_dir) if f.startswith("part_")]
    
    if not part_folders:
        part_folder = os.path.join(detail_view_dir, "part_1")
        os.makedirs(part_folder, exist_ok=True)
    else:
        part_folder = os.path.join(detail_view_dir, sorted(part_folders)[-1])
        if len(os.listdir(part_folder)) >= 300:
            part_num = len(part_folders) + 1
            part_folder = os.path.join(detail_view_dir, f"part_{part_num}")
            os.makedirs(part_folder, exist_ok=True)
    
    clean_product_name = re.sub(r'[\\/*?:"<>|]', "", product_name.replace(" ", "_"))
    
    timestamp = f"{year}_{month}_{day}_{hour_minute}"
    item_folder = os.path.join(part_folder, f"{timestamp}_{sku}_{clean_product_name}")
    os.makedirs(item_folder, exist_ok=True)
    
    image_folder = os.path.join(item_folder, "images")
    os.makedirs(image_folder, exist_ok=True)
    
    return item_folder, image_folder

def save_html_source(folder_path, driver, filename="html.txt"):
    """Lưu source HTML của trang hiện tại vào file."""
    html_content = driver.page_source
    file_path = os.path.join(folder_path, filename)
    with open(file_path, "w", encoding="utf-8") as file:
        file.write(html_content)
    print(f"Saved HTML: {file_path}")

def save_product_data(folder_path, products, actions):
    """Lưu dữ liệu sản phẩm từ list view và trả về danh sách các links."""
    image_folder = os.path.join(folder_path, "images")
    
    product_links = []


    for product in products:
        try:
            actions.move_to_element(product).perform()
            time.sleep(1)
            
            product_element = product.find_element(By.CSS_SELECTOR, "div.product-top.has > a, div.product-top > a")
            product_href = product_element.get_attribute('href')
            product_links.append(product_href)
            
            product_images = product.find_elements(By.CSS_SELECTOR, "div.product-top.has > a.product.photo.product-item-photo > div.item-xmedia > div.product-image-wrapper > div.image-thumbnail > img, div.product-top > a.product.photo.product-item-photo > div.item-xmedia > div.product-image-wrapper > div.image-thumbnail.product-image-hover > img")
            image_counters = {}
            
            for img in product_images:
                img_url = img.get_attribute("src")
                if "data:image" in img_url:  
                    img_url = img.get_attribute("data-src")
                    
                if not img_url or "data:image" in img_url:
                    continue
                
                alt_text = img.get_attribute("alt") or f"product_{len(product_links)}"
                clean_name = re.sub(r'[\\/*?:"<>|]', "", alt_text.replace(" ", "_").replace("-", "_"))
                
                if clean_name not in image_counters:
                    image_counters[clean_name] = 1
                else:
                    image_counters[clean_name] += 1
                
                image_name = f"{clean_name}_{image_counters[clean_name]}.jpg"
                image_path = os.path.join(image_folder, image_name)
                
                try:
                    response = requests.get(img_url)
                    if response.status_code == 200:
                        with open(image_path, "wb") as file:
                            file.write(response.content)
                        print(f"Saved Image: {image_path}")
                except Exception as e:
                    print(f"Error saving image {img_url}: {e}")
                    
        except Exception as e:
            print(f"Error processing product: {e}")
    
    json_path = os.path.join(folder_path, "data.json")
    with open(json_path, "w", encoding="utf-8") as json_file:
        json.dump(product_links, json_file, ensure_ascii=False, indent=4)
    print(f"Saved JSON: {json_path}")
    
    return product_links

def scrape_product_detail(driver, product_url, paths):
    """
    Lấy chi tiết sản phẩm từ trang detail và lưu vào thư mục tương ứng.
    Cập nhật list_done.csv và detail_attempt.csv
    """
    try:
        driver.get(product_url)
        time.sleep(10)
        
        sku_value = driver.find_element(By.CSS_SELECTOR, "div.product.attribute.sku").text.strip()
        name_value = driver.find_element(By.CSS_SELECTOR, "div.product-name-label h1").text.strip()
        
        if ":" in sku_value:
            sku_value = sku_value.split(":")[1].strip()
        
        try:
            sale_value = driver.find_element(By.CSS_SELECTOR, "span.product-label.sale-label span").text
        except:
            sale_value = None
        
        product_info_price = driver.find_element(By.CSS_SELECTOR, "div.product-info-price")
        try:
            special_price_value = product_info_price.find_element(By.CSS_SELECTOR,"span.special-price").text
            old_price_value = product_info_price.find_element(By.CSS_SELECTOR,"span.old-price").text
        except:
            old_price_value = product_info_price.text
            special_price_value = None
        
        size_elements = driver.find_elements(By.CSS_SELECTOR, "div.swatch-option.text")
        sizes_value = "_".join([size.text for size in size_elements])
        
        ##### MÔ TẢ #####
        try:
            description_element = driver.find_element(By.CSS_SELECTOR, "div.value.std")
            description_value = description_element.text.replace("\n", " _ ")
            time.sleep(2)
        except:
            description_value = None
        
        print(f"Mô tả: {description_value}")
        
        image_urls = []
        image_elements = driver.find_elements(By.CSS_SELECTOR, "div.product.item-image.imgzoom img")
        for image_element in image_elements:
            image_url = image_element.get_attribute('src')
            if image_url:
                image_urls.append(image_url)
        
        item_folder, image_folder = create_detail_directory(
            paths['detail_view_dir'], 
            sku_value, 
            name_value
        )
        
        save_html_source(item_folder, driver)
        
        product_data = {
            "sku": sku_value,
            "name": name_value,
            "sale": sale_value,
            "old_price": old_price_value,
            "special_price": special_price_value,
            "sizes": sizes_value,
            "description": description_value,
            "url": product_url,
            "images": image_urls
        }
        
        json_path = os.path.join(item_folder, "data.json")
        with open(json_path, "w", encoding="utf-8") as json_file:
            json.dump(product_data, json_file, ensure_ascii=False, indent=4)
        
        for idx, image_url in enumerate(image_urls, start=1):
            image_name = f"{sku_value}_{idx:02}.jpg"
            image_path = os.path.join(image_folder, image_name)
            
            try:
                response = requests.get(image_url)
                if response.status_code == 200:
                    with open(image_path, "wb") as file:
                        file.write(response.content)
                    print(f"Saved Image: {image_path}")
            except Exception as e:
                print(f"Error saving image {image_url}: {e}")
        
        append_csv(paths['list_done_path'], [product_url])
        
        detail_attempt_exists = os.path.isfile(paths['detail_attempt_path']) and os.path.getsize(paths['detail_attempt_path']) > 0
        
        with open(paths['detail_attempt_path'], "a", encoding="utf-8", newline='') as f:
            fieldnames = ["sku", "name", "sale", "old_price", "special_price", "sizes", "description", "url", "images"]
            writer = csv.DictWriter(f, fieldnames=fieldnames)
            
            if not detail_attempt_exists:
                writer.writeheader()
            
            data_to_write = product_data.copy()
            data_to_write["images"] = json.dumps(product_data["images"], ensure_ascii=False)
            
            writer.writerow(data_to_write)
        
        print(f"Successfully scraped product: {name_value} ({sku_value})")
        return True
        
    except Exception as e:
        print(f"Error scraping {product_url}: {e}")
        return False

##### CHẶN THÔNG BÁO #####

options = Options()
options.add_argument("--disable-infobars")
options.add_argument("--disable-notifications")
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option('useAutomationExtension', False)
prefs = {
    "credentials_enable_service": False, 
    "profile.password_manager_enabled": False, 
}
options.add_experimental_option("prefs", prefs)

##### MAIN FUNCTION #####

def crawl_links():
    """
    Hàm thực hiện việc cào tất cả các links sản phẩm trước
    """
    paths = initialize_directory_structure(base_save_dir, web_url)
    
    driver = webdriver.Chrome(options=options)
    driver.maximize_window()
    time.sleep(5)
    
    driver.get(web_url)
    time.sleep(5)
    
    try: 
        close_button = driver.find_element(By.CSS_SELECTOR, "div[id='close-button-1454703513200']")
        actions = ActionChains(driver)
        actions.move_to_element(close_button).click().perform()
    except:
        print("Không có thông báo popup")
    
    actions = ActionChains(driver)
    
    menu_button = driver.find_element(By.CSS_SELECTOR, "div.action-menu-responsive")
    actions.move_to_element(menu_button).perform()
    time.sleep(3)
    
    categorys = driver.find_elements(By.CSS_SELECTOR, "li.static-menu.level0 > a, li.category-menu.level0 > a, li.level1 > a")
    keywords = ["nu", "dam", "vay", "ao", "quan", "phu-kien", "giay", "tui", "trang-suc", "outlet"]
    # keywords = ["dam", "vay"]
    pattern = r"/([^/]+?)(?:\.html)?$"
    
    for category in categorys:
        category_href = category.get_attribute('href')
        
        if any(keyword in category_href for keyword in keywords):
            match = re.search(pattern, category_href)
            category_name = match.group(1) if match else "unknown"
            
            print(f"Opening: {category_href} (Category: {category_name})")
            
            driver.execute_script(f"window.open('{category_href}');")
            driver.switch_to.window(driver.window_handles[-1])
            time.sleep(2)
            
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(2)
            
            while True:
                try:
                    pagination = driver.find_element(By.CSS_SELECTOR, 
                        "div.bottom-pagination > div.toolbar.toolbar-products > div.pages.list-inline > ul.pagination > li.item.current > a, " +
                        "div.pager > div.pages.list-inline > ul.pagination > li.item.current > a")
                    pagination_text = pagination.text
                except:
                    pagination_text = "1"
                
                print(f"Processing: {category_name} - Page {pagination_text}")
                
                category_dir, _ = create_collection_directory(paths['list_view_dir'], category_name, pagination_text)
                
                save_html_source(category_dir, driver)
                
                main = driver.find_element(By.CSS_SELECTOR, "ol.products.list.items.product-items.row")
                products = main.find_elements(By.CSS_SELECTOR, 
                    "li.item.product.product-item-info.product-item.col-xs-12.col-md-6, " +
                    "li.item.product.product-item-info.product-item.col-12.col-xs-6.col-md-4.col-lg-3")
                
                product_links = save_product_data(category_dir, products, actions)
                
                update_product_links(paths, product_links)
                
                try:
                    next_page = driver.find_element(By.CSS_SELECTOR, 
                        "div.bottom-pagination > div.toolbar.toolbar-products > div.pages.list-inline > ul.pagination > li.item.pages-item-next, " +
                        "div.pager > div.pages.list-inline > ul.pagination > li.item.pages-item-next")
                    
                    actions.move_to_element(next_page).click().perform()
                    time.sleep(3) 
                except:
                    print(f"Đã hoàn thành cào danh mục: {category_name}")
                    driver.close()
                    driver.switch_to.window(driver.window_handles[0])
                    break
    
    print("Hoàn thành quá trình cào tất cả links sản phẩm")
    driver.quit()
    
    return paths

def crawl_details(paths):
    """
    Hàm thực hiện việc cào chi tiết tất cả các sản phẩm từ list_add.csv
    """

    list_add_path = paths['list_add_path']
    list_done_path = paths['list_done_path']
    
    links_to_scrape = load_csv(list_add_path)
    links_done = load_csv(list_done_path)
    
    links_to_scrape = links_to_scrape - links_done
    
    if not links_to_scrape:
        print("Không có links mới cần cào chi tiết")
        return
    
    print(f"Bắt đầu cào chi tiết cho {len(links_to_scrape)} sản phẩm mới")
    
    driver = webdriver.Chrome(options=options)
    driver.maximize_window()
    time.sleep(5)
    
    for i, link in enumerate(links_to_scrape):
        print(f"Cào chi tiết sản phẩm {i+1}/{len(links_to_scrape)}: {link}")
        success = scrape_product_detail(driver, link, paths)
        
        if success:
            print(f"Hoàn thành cào chi tiết: {link}")
        else:
            print(f"Thất bại khi cào chi tiết: {link}")
        
        time.sleep(1.5)
    
    print(f"Hoàn thành cào chi tiết cho {len(links_to_scrape)} sản phẩm")
    driver.quit()

def main():
    """
    Hàm chính thực hiện quá trình cào: 
    1. Cào tất cả links sản phẩm từ categories
    2. Cào tất cả links sản phẩm từ collections
    3. Sau đó cào chi tiết các sản phẩm
    """
    print("Bắt đầu quá trình cào dữ liệu...")
    
    print("=== PHASE 1: Crawling category ===")
    paths = crawl_links()
    
    print("=== PHASE 2: Crawling collection ===")
    crawl_collections()
    
    print("=== PHASE 3: Crawling product details ===")
    crawl_details(paths)
    
    print("Hoàn thành toàn bộ quá trình cào dữ liệu!")

if __name__ == "__main__":
    main()


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.1.3 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "c:\Users\lapto\AppData\Local\Programs\Python\Python311\Lib\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "c:\Users\lapto\AppData\Local\Programs\Python\Python311\Lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  File "c:\Users\lapto\AppData\Local\Programs\Python\Python311\Lib\site-packages\ipykernel\kernelapp.py", 

AttributeError: _ARRAY_API not found


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.1.3 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "c:\Users\lapto\AppData\Local\Programs\Python\Python311\Lib\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "c:\Users\lapto\AppData\Local\Programs\Python\Python311\Lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  File "c:\Users\lapto\AppData\Local\Programs\Python\Python311\Lib\site-packages\ipykernel\kernelapp.py", 

AttributeError: _ARRAY_API not found

Bắt đầu quá trình cào dữ liệu...
=== PHASE 1: Crawling category ===
Initialized directory structure for elise
Không có thông báo popup
Opening: http://store.elise.vn/thoi-trang-nu (Category: thoi-trang-nu)
Processing: thoi-trang-nu - Page 1
Created collection directory: E:\INTERN\PORTFOLIO\AUTOMATION\SYSTEM\data\elise\2025\05\20\09_49\list_view\thoi-trang-nu_1
Saved HTML: E:\INTERN\PORTFOLIO\AUTOMATION\SYSTEM\data\elise\2025\05\20\09_49\list_view\thoi-trang-nu_1\html.txt
Saved Image: E:\INTERN\PORTFOLIO\AUTOMATION\SYSTEM\data\elise\2025\05\20\09_49\list_view\thoi-trang-nu_1\images\ĐẦM_XỐP_TRẮNG_NHĂN_CỔ_BẺ_ĐÍNH_HOA_1.jpg
Saved Image: E:\INTERN\PORTFOLIO\AUTOMATION\SYSTEM\data\elise\2025\05\20\09_49\list_view\thoi-trang-nu_1\images\ĐẦM_XỐP_TRẮNG_NHĂN_CỔ_BẺ_ĐÍNH_HOA_2.jpg
Saved Image: E:\INTERN\PORTFOLIO\AUTOMATION\SYSTEM\data\elise\2025\05\20\09_49\list_view\thoi-trang-nu_1\images\ĐẦM_TƠ_ĐEN_VÂN_NỔI_BUỘC_NƠ_CỔ_1.jpg
Saved Image: E:\INTERN\PORTFOLIO\AUTOMATION\SYSTEM\data\elise\2025\05\20

# PANTIO

In [ ]:
import os
import time
import requests
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.options import Options

options = Options()
options.add_argument("--disable-infobars")
options.add_argument("--disable-notifications")
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option('useAutomationExtension', False)
prefs = {
    "credentials_enable_service": False, 
    "profile.password_manager_enabled": False, 
}
options.add_experimental_option("prefs", prefs)

# URL trang web
web_url = "https://www.pantio.vn/"
driver = webdriver.Chrome(options=options)
driver.maximize_window()
driver.get(web_url)

categorys = driver.find_elements(By.CSS_SELECTOR, "li.megamenu a, li.dropdown a")

if categorys:
    category = categorys[1] 
    category_href = category.get_attribute('href')

    if 'collections' in category_href:
        pattern = r"collections/([^/?#]+)"

        match = re.search(pattern, category_href)
        if match:
            category_name = match.group(1).replace("-", "_")
            print(category_name) 

        print(f"Opening: {category_href}")

        driver.execute_script(f"window.open('{category_href}');")
        driver.switch_to.window(driver.window_handles[-1])
        time.sleep(10)  

        products = driver.find_elements(By.CSS_SELECTOR, "div.grid__item.large--one-third.medium--one-third.small--one-half.md-pd-left15")
        product_links = []

        os.makedirs("images", exist_ok=True)

        for product in products:
            actions = ActionChains(driver)
            actions.move_to_element(product).perform()
            time.sleep(5)
            
            product_element = product.find_element(By.CSS_SELECTOR, "div.product-img a")
            product_href = product_element.get_attribute('href')
            print(product_href)
            product_links.append(product_href)

            product_images = product.find_elements(By.CSS_SELECTOR, "div.product-img img")
            image_counters = {}

            for img in product_images:
                img_url = img.get_attribute("src")  
                alt_text = img.get_attribute("alt")  

                clean_name = alt_text.replace(" ", "_").replace("-", "_")

                if clean_name not in image_counters:
                    image_counters[clean_name] = 1  
                else:
                    image_counters[clean_name] += 1 

                filename = os.path.join("images", f"{clean_name}_{image_counters[clean_name]}.jpg")

                response = requests.get(img_url)
                if response.status_code == 200:
                    with open(filename, "wb") as file:
                        file.write(response.content)
                    print(f"Đã lưu: {filename}")
        while True:
            try:
                pagination = driver.find_element(By.CSS_SELECTOR, 'span.page.page-node.current')
                pagination_text = pagination.text
            except:
                pagination_text = "1"
            print(pagination_text)
            try:
                actions = ActionChains(driver)
                next_page = driver.find_element(By.CSS_SELECTOR, 'span.nextPage')
                actions.move_to_element(next_page).click().perform()
                time.sleep(5)
            except:
                driver.close()
                driver.switch_to.window(driver.window_handles[0])
                break


In [ ]:
import os
import time
import re
import random
import requests
from datetime import datetime
import pandas as pd
import zipfile
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import TimeoutException
from bs4 import BeautifulSoup
import json
import csv

##### PATH + LINK #####

base_save_dir = r'E:\INTERN\PORTFOLIO\AUTOMATION\SYSTEM\data'
web_url = "https://www.pantio.vn/"

##### FUNCTION TẠO THƯ MỤC #####

def extract_website_name(url):
    """
    Lấy tên website từ URL, ví dụ: 'https://www.pantio.vn/' -> 'pantio'
    """
    match = re.search(r"https?://(?:www\.)?([^./]+)", url)
    return match.group(1) if match else "unknown"

start_time = datetime.now()
year = start_time.strftime("%Y")
month = start_time.strftime("%m")
day = start_time.strftime("%d")
hour_minute = start_time.strftime("%H_%M")

def initialize_directory_structure(base_save_dir, web_url):
    """
    Tạo cấu trúc thư mục cơ bản theo yêu cầu:
    website > năm > tháng > ngày > giờ_phút > list_view | detail_view | CSVs
    """
    website_name = extract_website_name(web_url)
    
    website_dir = os.path.join(base_save_dir, website_name)
    os.makedirs(website_dir, exist_ok=True)
    
    list_info_all_path = os.path.join(website_dir, "list_info_all.csv")
    if not os.path.exists(list_info_all_path):
        with open(list_info_all_path, 'w', encoding='utf-8', newline='') as f:
            writer = csv.writer(f)
            writer.writerow(["url"])  
    
    year_dir = os.path.join(website_dir, year)
    month_dir = os.path.join(year_dir, month)
    day_dir = os.path.join(month_dir, day)
    time_dir = os.path.join(day_dir, hour_minute)
    
    os.makedirs(year_dir, exist_ok=True)
    os.makedirs(month_dir, exist_ok=True)
    os.makedirs(day_dir, exist_ok=True)
    os.makedirs(time_dir, exist_ok=True)
    
    list_view_dir = os.path.join(time_dir, "list_view")
    detail_view_dir = os.path.join(time_dir, "detail_view")
    
    os.makedirs(list_view_dir, exist_ok=True)
    os.makedirs(detail_view_dir, exist_ok=True)
    
    list_attempt_path = os.path.join(time_dir, "list_attempt.csv")
    list_add_path = os.path.join(time_dir, "list_add.csv")
    list_done_path = os.path.join(time_dir, "list_done.csv")
    detail_attempt_path = os.path.join(time_dir, "detail_attempt.csv")
    
    for path in [list_attempt_path, list_add_path, list_done_path]:
        if not os.path.exists(path):
            with open(path, 'w', encoding='utf-8', newline='') as f:
                writer = csv.writer(f)
                writer.writerow(["url"]) 
    
    if not os.path.exists(detail_attempt_path):
        with open(detail_attempt_path, 'w', encoding='utf-8', newline='') as f:
            writer = csv.writer(f)
            writer.writerow(["sku", "name", "original_price", "sale_price", "colors", "sizes", "description", "url", "images"])
    
    print(f"Initialized directory structure for {website_name}")
    
    return {
        'website_dir': website_dir,
        'time_dir': time_dir,
        'list_view_dir': list_view_dir,
        'detail_view_dir': detail_view_dir,
        'list_attempt_path': list_attempt_path,
        'list_add_path': list_add_path,
        'list_done_path': list_done_path,
        'detail_attempt_path': detail_attempt_path,
        'list_info_all_path': list_info_all_path
    }

def load_csv(file_path):
    """Hàm để đọc dữ liệu từ file CSV và trả về một set."""
    if not os.path.exists(file_path):
        return set()
    
    with open(file_path, mode='r', encoding='utf-8') as file:
        reader = csv.reader(file)
        next(reader, None)  
        return {row[0] for row in reader if row and row[0] != "url"}

def append_csv(file_path, data):
    """Hàm để lưu dữ liệu vào file CSV theo kiểu append."""
    file_exists = os.path.isfile(file_path) and os.path.getsize(file_path) > 0
    
    with open(file_path, "a", encoding="utf-8", newline='') as f:
        writer = csv.writer(f)
        if not file_exists:
            writer.writerow(["url"])
        for item in data:
            writer.writerow([item])

def update_product_links(paths, product_links):
    """
    Cập nhật các file CSV theo quy trình:
    - So sánh list_attempt với list_info_all để tìm ra các links mới (list_add)
    - Thêm các links mới vào list_info_all và list_add
    - Thêm tất cả các links vào list_attempt
    """
    list_info_all = load_csv(paths['list_info_all_path'])
    list_attempt = set(product_links) 
    
    list_add = list_attempt - list_info_all
    
    if list_add:
        append_csv(paths['list_info_all_path'], list_add)
        append_csv(paths['list_add_path'], list_add)
    
    append_csv(paths['list_attempt_path'], list_attempt)
    
    print(f"Updated CSV files:")
    print(f"- Added {len(list_add)} new links to list_add.csv")
    print(f"- Added {len(list_attempt)} links to list_attempt.csv")
    
    return list_add

def create_category_directory(list_view_dir, category_name, pagination_text):
    """
    Tạo thư mục category_{pagination} trong list_view
    """
    category_dir = os.path.join(list_view_dir, f"{category_name}_{pagination_text}")
    images_dir = os.path.join(category_dir, "images")
    
    os.makedirs(category_dir, exist_ok=True)
    os.makedirs(images_dir, exist_ok=True)
    
    print(f"Created category directory: {category_dir}")
    
    return category_dir, images_dir

def create_detail_directory(detail_view_dir, sku, product_name):
    """
    Tạo thư mục trong detail_view theo cấu trúc:
    part_X > {năm_tháng_ngày_giờ_phút}_{sku}_{name}
    """
    part_folders = [f for f in os.listdir(detail_view_dir) if f.startswith("part_")]
    
    if not part_folders:
        part_folder = os.path.join(detail_view_dir, "part_1")
        os.makedirs(part_folder, exist_ok=True)
    else:
        part_folder = os.path.join(detail_view_dir, sorted(part_folders)[-1])
        if len(os.listdir(part_folder)) >= 300:
            part_num = len(part_folders) + 1
            part_folder = os.path.join(detail_view_dir, f"part_{part_num}")
            os.makedirs(part_folder, exist_ok=True)
    
    clean_product_name = re.sub(r'[\\/*?:"<>|]', "", product_name.replace(" ", "_"))
    
    timestamp = f"{year}_{month}_{day}_{hour_minute}"
    item_folder = os.path.join(part_folder, f"{timestamp}_{sku}_{clean_product_name}")
    os.makedirs(item_folder, exist_ok=True)
    
    image_folder = os.path.join(item_folder, "images")
    os.makedirs(image_folder, exist_ok=True)
    
    return item_folder, image_folder

def save_html_source(folder_path, driver, filename="html.txt"):
    """Lưu source HTML của trang hiện tại vào file."""
    html_content = driver.page_source
    file_path = os.path.join(folder_path, filename)
    with open(file_path, "w", encoding="utf-8") as file:
        file.write(html_content)
    print(f"Saved HTML: {file_path}")

def save_product_data(folder_path, products, actions):
    """Lưu dữ liệu sản phẩm từ list view và trả về danh sách các links."""
    image_folder = os.path.join(folder_path, "images")
    
    product_links = []
    
    for product in products:
        try:
            actions.move_to_element(product).perform()
            time.sleep(1)
            
            product_element = product.find_element(By.CSS_SELECTOR, "div.product-img a")
            product_href = product_element.get_attribute('href')
            product_links.append(product_href)
            
            product_images = product.find_elements(By.CSS_SELECTOR, "div.product-img img")
            image_counters = {}
            
            for img in product_images:
                img_url = img.get_attribute("src")
                if "data:image" in img_url:  
                    img_url = img.get_attribute("data-src")
                    
                if not img_url or "data:image" in img_url:
                    continue
                
                alt_text = img.get_attribute("alt") or f"product_{len(product_links)}"
                clean_name = re.sub(r'[\\/*?:"<>|]', "", alt_text.replace(" ", "_").replace("-", "_"))
                
                if clean_name not in image_counters:
                    image_counters[clean_name] = 1
                else:
                    image_counters[clean_name] += 1
                
                image_name = f"{clean_name}_{image_counters[clean_name]}.jpg"
                image_path = os.path.join(image_folder, image_name)
                
                try:
                    response = requests.get(img_url)
                    if response.status_code == 200:
                        with open(image_path, "wb") as file:
                            file.write(response.content)
                        print(f"Saved Image: {image_path}")
                except Exception as e:
                    print(f"Error saving image {img_url}: {e}")
                    
        except Exception as e:
            print(f"Error processing product: {e}")
    
    json_path = os.path.join(folder_path, "data.json")
    with open(json_path, "w", encoding="utf-8") as json_file:
        json.dump(product_links, json_file, ensure_ascii=False, indent=4)
    print(f"Saved JSON with {len(product_links)} product links: {json_path}")
    
    return product_links

def scrape_product_detail(driver, product_url, paths):
    """
    Lấy chi tiết sản phẩm từ trang detail của Pantio và lưu vào thư mục tương ứng.
    Cập nhật list_done.csv và detail_attempt.csv
    """
    try:
        driver.get(product_url)
        time.sleep(5)
        
        try: 
            sku_element = driver.find_element(By.CSS_SELECTOR, 'div.pro-sku span.sku-number')
            sku_value = sku_element.text.strip()
            time.sleep(1)
        except: 
            sku_value = f"unknown_sku_{int(time.time())}"
            print("Không tìm thấy SKU, sử dụng mã tạm thời")
        
        try:
            name_element = driver.find_element(By.CSS_SELECTOR, 'h1[itemprop="name"]')
            name_value = name_element.text.strip()
            time.sleep(1)
        except: 
            name_value = f"Unknown Product {int(time.time())}"
            print("Không tìm thấy tên sản phẩm, sử dụng tên tạm thời")
        
        try:
            price_element = driver.find_element(By.CSS_SELECTOR, 'div.pro-price.clearfix span.original-price.ComparePrice')
            original_price_value = price_element.text.strip()
            
            sale_element = driver.find_element(By.CSS_SELECTOR, 'div.pro-price.clearfix span.current-price.ProductPrice')
            sale_price_value = sale_element.text.strip()
        except:
            try:
                price_element = driver.find_element(By.CSS_SELECTOR, 'div.pro-price.clearfix span.current-price.ProductPrice')
                original_price_value = price_element.text.strip()
                sale_price_value = None
            except:
                original_price_value = None
                sale_price_value = None
                print("Không tìm thấy thông tin giá")
        
        try:
            swatch_element = driver.find_element(By.ID, 'variant-swatch-0')
            color_elements = swatch_element.find_elements(By.CSS_SELECTOR, '.swatch-element')
            colors_value = [element.get_attribute('data-value') for element in color_elements]
            colors_string = "_".join(colors_value)
        except:
            colors_value = []
            colors_string = None
            print("Không tìm thấy thông tin màu sắc")
        
        try: 
            swatch_element = driver.find_element(By.ID, 'variant-swatch-1')
            size_elements = swatch_element.find_elements(By.CSS_SELECTOR, '.swatch-element')
            sizes_value = [element.get_attribute('data-value') for element in size_elements]
            sizes_string = "_".join(sizes_value)
        except:
            sizes_value = []
            sizes_string = None
            print("Không tìm thấy thông tin kích thước")
        
        try:
            description_element = driver.find_element(By.CSS_SELECTOR, 'div.pro-short-desc')
            description_value = description_element.text.strip().replace("\n", " _ ")
            time.sleep(1)
        except:
            description_value = None
            print("Không tìm thấy mô tả sản phẩm")
        
        image_urls = []
        try:
            list_image_element = driver.find_element(By.CSS_SELECTOR, 'div.slick-list.draggable')
            image_elements = list_image_element.find_elements(By.CSS_SELECTOR, 'li.product-gallery.slick-slide img.product-image-feature2')
            
            for image_element in image_elements:
                image_url = image_element.get_attribute('src')
                if image_url and "data:image" not in image_url:
                    image_urls.append(image_url)
                    
            if not image_urls:
                image_elements = driver.find_elements(By.CSS_SELECTOR, 'div.product-gallery img')
                for image_element in image_elements:
                    image_url = image_element.get_attribute('src')
                    if image_url and "data:image" not in image_url:
                        image_urls.append(image_url)
        except Exception as e:
            print(f"Lỗi khi lấy ảnh sản phẩm: {e}")
        
        item_folder, image_folder = create_detail_directory(
            paths['detail_view_dir'], 
            sku_value, 
            name_value
        )
        
        save_html_source(item_folder, driver)
        
        product_data = {
            "sku": sku_value,
            "name": name_value,
            "original_price": original_price_value,
            "sale_price": sale_price_value,
            "colors": colors_string,
            "sizes": sizes_string,
            "description": description_value,
            "url": product_url,
            "images": image_urls
        }
        
        json_path = os.path.join(item_folder, "data.json")
        with open(json_path, "w", encoding="utf-8") as json_file:
            json.dump(product_data, json_file, ensure_ascii=False, indent=4)
        
        for idx, image_url in enumerate(image_urls, start=1):
            image_name = f"{sku_value}_{idx:02}.jpg"
            image_path = os.path.join(image_folder, image_name)
            
            try:
                response = requests.get(image_url)
                if response.status_code == 200:
                    with open(image_path, "wb") as file:
                        file.write(response.content)
                    print(f"Saved Image: {image_path}")
            except Exception as e:
                print(f"Error saving image {image_url}: {e}")
        
        append_csv(paths['list_done_path'], [product_url])
        
        detail_attempt_exists = os.path.isfile(paths['detail_attempt_path']) and os.path.getsize(paths['detail_attempt_path']) > 0
        
        with open(paths['detail_attempt_path'], "a", encoding="utf-8", newline='') as f:
            fieldnames = ["sku", "name", "original_price", "sale_price", "colors", "sizes", "description", "url", "images"]
            writer = csv.DictWriter(f, fieldnames=fieldnames)
            
            if not detail_attempt_exists:
                writer.writeheader()
            
            data_to_write = product_data.copy()
            data_to_write["images"] = json.dumps(product_data["images"], ensure_ascii=False)
            
            writer.writerow(data_to_write)
        
        print(f"Successfully scraped product: {name_value} ({sku_value})")
        return True
        
    except Exception as e:
        print(f"Error scraping {product_url}: {e}")
        return False

##### CHẶN THÔNG BÁO #####

options = Options()
options.add_argument("--disable-infobars")
options.add_argument("--disable-notifications")
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option('useAutomationExtension', False)
prefs = {
    "credentials_enable_service": False, 
    "profile.password_manager_enabled": False, 
}
options.add_experimental_option("prefs", prefs)

##### MAIN FUNCTION #####

def crawl_links():
    """
    Hàm thực hiện việc cào tất cả các links sản phẩm từ Pantio
    """
    paths = initialize_directory_structure(base_save_dir, web_url)
    
    driver = webdriver.Chrome(options=options)
    driver.maximize_window()
    time.sleep(5)
    
    driver.get(web_url)
    time.sleep(5)
    
    actions = ActionChains(driver)
    
    categorys = driver.find_elements(By.CSS_SELECTOR, "li.megamenu a, li.dropdown a")
    
    for index, category in enumerate(categorys):
        if index >= 3:
            break
        category_href = category.get_attribute('href')
        if 'collections' in category_href:
            pattern = r"collections/([^/?#]+)"
            match = re.search(pattern, category_href)
            if match:
                category_name = match.group(1).replace("-", "_")
            else:
                category_name = re.sub(r'[\\/*?:"<>|]', "", category_href.replace(" ", "_").replace("-", "_"))
            
            print(f"Opening: {category_href} (Category: {category_name})")
            
            driver.execute_script(f"window.open('{category_href}');")
            driver.switch_to.window(driver.window_handles[-1])
            time.sleep(5)
            
            has_next_page = True
            
            while has_next_page:
                try:
                    pagination = driver.find_element(By.CSS_SELECTOR, 'span.page.page-node.current')
                    pagination_text = pagination.text
                except:
                    pagination_text = "1"
                
                print(f"Processing: {category_name} - Page {pagination_text}")
                
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                time.sleep(3)
                
                category_dir, _ = create_category_directory(paths['list_view_dir'], category_name, pagination_text)
                
                save_html_source(category_dir, driver)
                
                products = driver.find_elements(By.CSS_SELECTOR, "div.grid__item.large--one-third.medium--one-third.small--one-half.md-pd-left15")
                
                if not products:
                    products = driver.find_elements(By.CSS_SELECTOR, "div.product-item")
                
                if products:
                    product_links = save_product_data(category_dir, products, actions)
                    
                    update_product_links(paths, product_links)
                else:
                    print(f"Không tìm thấy sản phẩm nào trong trang {pagination_text}")
                
                try:
                    next_page = driver.find_element(By.CSS_SELECTOR, "span.nextPage")
                    actions.move_to_element(next_page).click().perform()
                    time.sleep(5) 
                except Exception as e:
                    print(f"Không tìm thấy nút next page hoặc đã hết trang: {e}")
                    has_next_page = False
            
            driver.close()
            driver.switch_to.window(driver.window_handles[0])
            
    driver.quit()
    
    return paths

def crawl_details(paths):
    """
    Hàm thực hiện việc cào chi tiết tất cả các sản phẩm từ list_add.csv
    """
    list_add_path = paths['list_add_path']
    list_done_path = paths['list_done_path']
    
    links_to_scrape = load_csv(list_add_path)
    links_done = load_csv(list_done_path)
    
    links_to_scrape = links_to_scrape - links_done
    
    if not links_to_scrape:
        print("Không có links mới cần cào chi tiết")
        return
    
    print(f"Bắt đầu cào chi tiết cho {len(links_to_scrape)} sản phẩm mới")
    
    driver = webdriver.Chrome(options=options)
    driver.maximize_window()
    time.sleep(2)
    
    for i, link in enumerate(links_to_scrape):
        print(f"Cào chi tiết sản phẩm {i+1}/{len(links_to_scrape)}: {link}")
        success = scrape_product_detail(driver, link, paths)
        
        if success:
            print(f"Hoàn thành cào chi tiết: {link}")
        else:
            print(f"Thất bại khi cào chi tiết: {link}")
        
        time.sleep(2)
    
    print(f"Hoàn thành cào chi tiết cho {len(links_to_scrape)} sản phẩm")
    driver.quit()

def main():
    """
    Hàm chính thực hiện quá trình cào: 
    1. Cào tất cả links sản phẩm từ categories
    2. Sau đó cào chi tiết các sản phẩm
    """
    print("Bắt đầu quá trình cào dữ liệu...")
    
    print("=== PHASE 1: Crawling category ===")
    paths = crawl_links()
    
    
    print("=== PHASE 2: Crawling product details ===")
    crawl_details(paths)
    
    print("Hoàn thành toàn bộ quá trình cào dữ liệu!")

if __name__ == "__main__":
    main()

# IVYMODA

In [ ]:
import os
import time
import requests
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.options import Options

options = Options()
options.add_argument("--disable-infobars")
options.add_argument("--disable-notifications")
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option('useAutomationExtension', False)
prefs = {
    "credentials_enable_service": False, 
    "profile.password_manager_enabled": False, 
}
options.add_experimental_option("prefs", prefs)

# URL trang web
web_url = "https://ivymoda.com/"
driver = webdriver.Chrome(options=options)
driver.maximize_window()
driver.get(web_url)


# Xóa thông báo 
try:  
    close_button = driver.find_element(By.CSS_SELECTOR, "div.popup-content div.action-close")
    actions = ActionChains(driver)
    actions.move_to_element(close_button).click().perform()
except:
        print("Không có thông báo nào.")  


actions = ActionChains(driver)
menu_element = driver.find_element(By.CSS_SELECTOR, "ul.menu")
categorys = menu_element.find_elements(By.TAG_NAME, "a")
for category in categorys: 
    category_href = category.get_attribute('href')
    if 'danh-muc' in category_href or 'lookbook' in category_href:

        pattern = r"(?:danh-muc|lookbook)/([^/?#]+)"
        match = re.search(pattern, category_href)
        if match:
            category_name = match.group(1).replace("-", "_")

        print(category_name)
        print(category_href)
        # driver.execute_script(f"window.open('{category_href}');")
        # driver.switch_to.window(driver.window_handles[-1])
        # time.sleep(10)

        # os.makedirs("images", exist_ok=True)
        # products = driver.find_elements(By.CSS_SELECTOR, "div.item-cat-product div.product div.thumb-product a")
        # for product in products: 
        #     actions.move_to_element(product).perform()
        #     product_href = product.get_attribute('href')
        #     print(product_href)

        #     product_images = product.find_elements(By.CSS_SELECTOR, "div.item-cat-product div.product div.thumb-product img")
        #     image_counters = {}
        #     for img in product_images:
        #         img_url = img.get_attribute("src") or img.get_attribute("data-original")
        #         alt_text = img.get_attribute("alt")  

        #         clean_name = alt_text.replace(" ", "_").replace("-", "_")

        #         if clean_name not in image_counters:
        #             image_counters[clean_name] = 1  
        #         else:
        #             image_counters[clean_name] += 1 

        #         filename = os.path.join("images", f"{clean_name}_{image_counters[clean_name]}.jpg")

        #         response = requests.get(img_url)
        #         if response.status_code == 200:
        #             with open(filename, "wb") as file:
        #                 file.write(response.content)
        #             print(f"Đã lưu: {filename}")

        # while True:
        #     try:
        #         pagination = driver.find_element("xpath", '//*[@id="products_active_ts"]')
        #         pagination_text = pagination.text
        #     except:
        #         pagination_text = "1"
        #     print(pagination_text)
        #     try:
        #         actions = ActionChains(driver)
        #         next_page = driver.find_element("xpath", '//li[@class="last-page"]/preceding-sibling::li[1]')
        #         actions.move_to_element(next_page).click().perform()
        #         time.sleep(5)
        #     except:
        #         driver.close()
        #         driver.switch_to.window(driver.window_handles[0])
        #         break

In [ ]:
import os
import time
import re
import random
import requests
from datetime import datetime
import pandas as pd
import zipfile
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import TimeoutException
from bs4 import BeautifulSoup
import json
import csv

##### PATH + LINK #####

base_save_dir = r'E:\INTERN\PORTFOLIO\AUTOMATION\SYSTEM\data'
web_url = "https://ivymoda.com/"

##### FUNCTION TẠO THƯ MỤC #####

def extract_website_name(url):
    """
    Lấy tên website từ URL, ví dụ: 'https://ivymoda.com/' -> 'ivymoda'
    """
    match = re.search(r"https?://(?:www\.)?([^./]+)", url)
    return match.group(1) if match else "unknown"

start_time = datetime.now()
year = start_time.strftime("%Y")
month = start_time.strftime("%m")
day = start_time.strftime("%d")
hour_minute = start_time.strftime("%H_%M")

def initialize_directory_structure(base_save_dir, web_url):
    """
    Tạo cấu trúc thư mục cơ bản theo yêu cầu:
    website > năm > tháng > ngày > giờ_phút > list_view | detail_view | CSVs
    """
    website_name = extract_website_name(web_url)
    
    website_dir = os.path.join(base_save_dir, website_name)
    os.makedirs(website_dir, exist_ok=True)
    
    list_info_all_path = os.path.join(website_dir, "list_info_all.csv")
    if not os.path.exists(list_info_all_path):
        with open(list_info_all_path, 'w', encoding='utf-8', newline='') as f:
            writer = csv.writer(f)
            writer.writerow(["url"])  
    
    year_dir = os.path.join(website_dir, year)
    month_dir = os.path.join(year_dir, month)
    day_dir = os.path.join(month_dir, day)
    time_dir = os.path.join(day_dir, hour_minute)
    
    os.makedirs(year_dir, exist_ok=True)
    os.makedirs(month_dir, exist_ok=True)
    os.makedirs(day_dir, exist_ok=True)
    os.makedirs(time_dir, exist_ok=True)
    
    list_view_dir = os.path.join(time_dir, "list_view")
    detail_view_dir = os.path.join(time_dir, "detail_view")
    
    os.makedirs(list_view_dir, exist_ok=True)
    os.makedirs(detail_view_dir, exist_ok=True)
    
    list_attempt_path = os.path.join(time_dir, "list_attempt.csv")
    list_add_path = os.path.join(time_dir, "list_add.csv")
    list_done_path = os.path.join(time_dir, "list_done.csv")
    detail_attempt_path = os.path.join(time_dir, "detail_attempt.csv")
    
    for path in [list_attempt_path, list_add_path, list_done_path]:
        if not os.path.exists(path):
            with open(path, 'w', encoding='utf-8', newline='') as f:
                writer = csv.writer(f)
                writer.writerow(["url"]) 
    
    if not os.path.exists(detail_attempt_path):
        with open(detail_attempt_path, 'w', encoding='utf-8', newline='') as f:
            writer = csv.writer(f)
            writer.writerow(["sku", "name", "original_price", "sale_price", "sizes", "colors", "description", "url", "images"])
    
    print(f"Initialized directory structure for {website_name}")
    
    return {
        'website_dir': website_dir,
        'time_dir': time_dir,
        'list_view_dir': list_view_dir,
        'detail_view_dir': detail_view_dir,
        'list_attempt_path': list_attempt_path,
        'list_add_path': list_add_path,
        'list_done_path': list_done_path,
        'detail_attempt_path': detail_attempt_path,
        'list_info_all_path': list_info_all_path
    }

def load_csv(file_path):
    """Hàm để đọc dữ liệu từ file CSV và trả về một set."""
    if not os.path.exists(file_path):
        return set()
    
    with open(file_path, mode='r', encoding='utf-8') as file:
        reader = csv.reader(file)
        next(reader, None)  # Skip header row
        return {row[0] for row in reader if row and row[0] != "url"}

def append_csv(file_path, data):
    """Hàm để lưu dữ liệu vào file CSV theo kiểu append."""
    file_exists = os.path.isfile(file_path) and os.path.getsize(file_path) > 0
    
    with open(file_path, "a", encoding="utf-8", newline='') as f:
        writer = csv.writer(f)
        if not file_exists:
            writer.writerow(["url"])
        for item in data:
            writer.writerow([item])

def update_product_links(paths, product_links):
    """
    Cập nhật các file CSV theo quy trình:
    - So sánh list_attempt với list_info_all để tìm ra các links mới (list_add)
    - Thêm các links mới vào list_info_all và list_add
    - Thêm tất cả các links vào list_attempt
    """
    list_info_all = load_csv(paths['list_info_all_path'])
    list_attempt = set(product_links) 
    
    list_add = list_attempt - list_info_all
    
    if list_add:
        append_csv(paths['list_info_all_path'], list_add)
        append_csv(paths['list_add_path'], list_add)
    
    append_csv(paths['list_attempt_path'], list_attempt)
    
    print(f"Updated CSV files:")
    print(f"- Added {len(list_add)} new links to list_add.csv")
    print(f"- Added {len(list_attempt)} links to list_attempt.csv")
    
    return list_add

def create_category_directory(list_view_dir, category_name, pagination_text):
    """
    Tạo thư mục category_{pagination} trong list_view
    """
    category_dir = os.path.join(list_view_dir, f"{category_name}_{pagination_text}")
    images_dir = os.path.join(category_dir, "images")
    
    os.makedirs(category_dir, exist_ok=True)
    os.makedirs(images_dir, exist_ok=True)
    
    print(f"Created category directory: {category_dir}")
    
    return category_dir, images_dir

def create_detail_directory(detail_view_dir, sku, product_name):
    """
    Tạo thư mục trong detail_view theo cấu trúc:
    part_X > {năm_tháng_ngày_giờ_phút}_{sku}_{name}
    """
    part_folders = [f for f in os.listdir(detail_view_dir) if f.startswith("part_")]
    
    if not part_folders:
        part_folder = os.path.join(detail_view_dir, "part_1")
        os.makedirs(part_folder, exist_ok=True)
    else:
        part_folder = os.path.join(detail_view_dir, sorted(part_folders)[-1])
        if len(os.listdir(part_folder)) >= 300:
            part_num = len(part_folders) + 1
            part_folder = os.path.join(detail_view_dir, f"part_{part_num}")
            os.makedirs(part_folder, exist_ok=True)
    
    clean_product_name = re.sub(r'[\\/*?:"<>|]', "", product_name.replace(" ", "_"))
    
    timestamp = f"{year}_{month}_{day}_{hour_minute}"
    item_folder = os.path.join(part_folder, f"{timestamp}_{sku}_{clean_product_name}")
    os.makedirs(item_folder, exist_ok=True)
    
    image_folder = os.path.join(item_folder, "images")
    os.makedirs(image_folder, exist_ok=True)
    
    return item_folder, image_folder

def save_html_source(folder_path, driver, filename="html.txt"):
    """Lưu source HTML của trang hiện tại vào file."""
    html_content = driver.page_source
    file_path = os.path.join(folder_path, filename)
    with open(file_path, "w", encoding="utf-8") as file:
        file.write(html_content)
    print(f"Saved HTML: {file_path}")

def save_product_data(folder_path, products, actions):
    """Lưu dữ liệu sản phẩm từ list view của IvyModa và trả về danh sách các links."""
    image_folder = os.path.join(folder_path, "images")
    
    product_links = []
    
    for product in products:
        try:
            actions.move_to_element(product).perform()
            
            # Lấy link sản phẩm từ IvyModa
            product_href = product.get_attribute('href')
            if product_href:
                product_links.append(product_href)
                
                # Lấy và lưu ảnh sản phẩm
                parent_product = product.find_element(By.XPATH, "./..")
                product_images = parent_product.find_elements(By.CSS_SELECTOR, "img")
                
                image_counters = {}
                
                for img in product_images:
                    img_url = img.get_attribute("src") or img.get_attribute("data-original")
                    if not img_url or "data:image" in img_url:
                        continue
                    
                    alt_text = img.get_attribute("alt") or f"product_{len(product_links)}"
                    clean_name = re.sub(r'[\\/*?:"<>|]', "", alt_text.replace(" ", "_").replace("-", "_"))
                    
                    if clean_name not in image_counters:
                        image_counters[clean_name] = 1
                    else:
                        image_counters[clean_name] += 1
                    
                    image_name = f"{clean_name}_{image_counters[clean_name]}.jpg"
                    image_path = os.path.join(image_folder, image_name)
                    
                    try:
                        response = requests.get(img_url)
                        if response.status_code == 200:
                            with open(image_path, "wb") as file:
                                file.write(response.content)
                            print(f"Saved Image: {image_path}")
                    except Exception as e:
                        print(f"Error saving image {img_url}: {e}")
        except Exception as e:
            print(f"Error processing product: {e}")
    
    # Lưu danh sách links vào file JSON
    json_path = os.path.join(folder_path, "data.json")
    with open(json_path, "w", encoding="utf-8") as json_file:
        json.dump(product_links, json_file, ensure_ascii=False, indent=4)
    print(f"Saved JSON with {len(product_links)} product links: {json_path}")
    
    return product_links

def scrape_product_detail(driver, product_url, paths):
    """
    Lấy chi tiết sản phẩm từ trang detail của IvyModa và lưu vào thư mục tương ứng.
    Cập nhật list_done.csv và detail_attempt.csv
    """
    try:
        driver.get(product_url)
        time.sleep(5)
        
        # Khởi tạo ActionChains
        actions = ActionChains(driver)
        
        # Lấy mã sản phẩm (SKU)
        try: 
            sku_element = driver.find_element(By.CSS_SELECTOR, 'div.product-detail__sub-info p span')
            sku_value = sku_element.text.strip()
            time.sleep(1)
        except Exception as e: 
            print(f"Error getting SKU: {e}")
            sku_value = f"unknown_sku_{int(time.time())}"
        
        # Lấy tên sản phẩm
        try:
            name_element = driver.find_element(By.CSS_SELECTOR, 'div.product-detail__information h1')
            name_value = name_element.text.strip()
            time.sleep(1)
        except Exception as e: 
            print(f"Error getting name: {e}")
            name_value = f"Unknown Product {int(time.time())}"
        
        # Lấy giá gốc và giá khuyến mãi
        try:
            price_element = driver.find_element(By.CSS_SELECTOR, 'div.product-detail__price del')
            original_price_value = price_element.text.strip()
            
            sale_element = driver.find_element(By.CSS_SELECTOR, 'div.product-detail__price b')
            sale_price_value = sale_element.text.strip()
        except:
            try:
                price_element = driver.find_element(By.CSS_SELECTOR, 'div.product-detail__price b')
                original_price_value = price_element.text.strip()
                sale_price_value = None
            except Exception as e:
                print(f"Error getting price: {e}")
                original_price_value = None
                sale_price_value = None
        
        # Lấy thông tin kích thước
        try:
            size_elements = driver.find_elements(By.CSS_SELECTOR, 'div.product-detail__size__input span.text-uppercase')
            sizes_value = [element.text for element in size_elements]
            sizes_string = "_".join(sizes_value)
        except Exception as e:
            print(f"Error getting sizes: {e}")
            sizes_value = []
            sizes_string = None
        
        # Lấy mô tả sản phẩm
        introduction_value = []
        try:
            description_buttons = driver.find_elements(By.CSS_SELECTOR, 'div.product-detail__tab div.product-detail__tab-header div.tab-item')
            for description_button in description_buttons: 
                actions.move_to_element(description_button).click().perform()
                time.sleep(2)
                
                # Thử nhấn nút "Xem thêm" nếu có
                try:
                    show_more_button = driver.find_element(By.CSS_SELECTOR, 'div.product-detail__tab div.product-detail__tab-body div.show-more a')
                    actions.move_to_element(show_more_button).click().perform()
                    time.sleep(2)
                except:
                    pass
                
                introduction_element = driver.find_element(By.CSS_SELECTOR, 'div.product-detail__tab-body div.tab-content.active.showContent')
                
                p_elements = introduction_element.find_elements(By.TAG_NAME, 'p')
                tr_elements = introduction_element.find_elements(By.TAG_NAME, 'tr')                
                all_elements = p_elements + tr_elements
                
                value = [element.text for element in all_elements]
                introduction_value.extend(value)
        except Exception as e:
            print(f"Error getting description: {e}")
        
        description_value = " _ ".join([text for text in introduction_value if text.strip()]) if introduction_value else None
        
        # Lấy thông tin màu sắc và hình ảnh
        color_values = []
        image_urls = []
        
        try: 
            # Trước tiên, lấy ảnh từ trang hiện tại
            image_elements_current = driver.find_elements(By.CSS_SELECTOR, 'div.product-detail__gallery img.lazyloaded, div.product-detail__gallery img')
            for image_element in image_elements_current:
                image_url = image_element.get_attribute('src') or image_element.get_attribute('data-src')
                if image_url and not "data:image" in image_url and image_url not in image_urls:
                    image_urls.append(image_url)
            
            # Sau đó kiểm tra các màu khác
            button_color_elements = driver.find_elements(By.CSS_SELECTOR, 'div.product-detail__color div.product-detail__color__input label span a')
            for button_color_element in button_color_elements:
                color_href = button_color_element.get_attribute('href')
                if not color_href:
                    continue
                
                print(f"Checking color variant: {color_href}")
                
                # Mở màu sản phẩm trong tab mới
                driver.execute_script(f"window.open('{color_href}');")
                driver.switch_to.window(driver.window_handles[-1])
                time.sleep(5)
                
                try:
                    color_element = driver.find_element(By.CSS_SELECTOR, 'div.product-detail__color p')
                    color_value = color_element.text
                    color_values.append(color_value)
                    
                    # Lấy ảnh của màu này
                    image_elements = driver.find_elements(By.CSS_SELECTOR, 'div.product-detail__gallery img.lazyloaded, div.product-detail__gallery img')
                    for image_element in image_elements:
                        image_url = image_element.get_attribute('src') or image_element.get_attribute('data-src')
                        if image_url and not "data:image" in image_url and image_url not in image_urls:
                            image_urls.append(image_url)
                except Exception as e:
                    print(f"Error getting color info: {e}")
                
                # Đóng tab màu sản phẩm
                driver.close()
                driver.switch_to.window(driver.window_handles[-1])
                time.sleep(1)
        except Exception as e:
            print(f"Error getting colors and images: {e}")
        
        colors_string = "_".join(color_values) if color_values else None
        
        # Tạo thư mục lưu trữ
        item_folder, image_folder = create_detail_directory(
            paths['detail_view_dir'], 
            sku_value, 
            name_value
        )
        
        # Lưu HTML source
        save_html_source(item_folder, driver)
        
        # Tạo object dữ liệu sản phẩm
        product_data = {
            "sku": sku_value,
            "name": name_value,
            "original_price": original_price_value,
            "sale_price": sale_price_value,
            "sizes": sizes_string,
            "colors": colors_string,
            "description": description_value,
            "url": product_url,
            "images": image_urls
        }
        
        # Lưu dữ liệu vào file JSON
        json_path = os.path.join(item_folder, "data.json")
        with open(json_path, "w", encoding="utf-8") as json_file:
            json.dump(product_data, json_file, ensure_ascii=False, indent=4)
        
        # Tải và lưu các hình ảnh
        for idx, image_url in enumerate(image_urls, start=1):
            image_name = f"{sku_value}_{idx:02}.jpg"
            image_path = os.path.join(image_folder, image_name)
            
            try:
                response = requests.get(image_url)
                if response.status_code == 200:
                    with open(image_path, "wb") as file:
                        file.write(response.content)
                    print(f"Saved Image: {image_path}")
            except Exception as e:
                print(f"Error saving image {image_url}: {e}")
        
        # Cập nhật list_done.csv
        append_csv(paths['list_done_path'], [product_url])
        
        # Cập nhật detail_attempt.csv
        detail_attempt_exists = os.path.isfile(paths['detail_attempt_path']) and os.path.getsize(paths['detail_attempt_path']) > 0
        
        with open(paths['detail_attempt_path'], "a", encoding="utf-8", newline='') as f:
            # Sử dụng DictWriter để đảm bảo định dạng CSV đúng
            fieldnames = ["sku", "name", "original_price", "sale_price", "sizes", "colors", "description", "url", "images"]
            writer = csv.DictWriter(f, fieldnames=fieldnames)
            
            # Nếu file chưa tồn tại hoặc rỗng, ghi header
            if not detail_attempt_exists:
                writer.writeheader()
            
            # Chuyển đổi list image_urls thành string để lưu trong CSV
            data_to_write = product_data.copy()
            data_to_write["images"] = json.dumps(product_data["images"], ensure_ascii=False)
            
            # Ghi dữ liệu
            writer.writerow(data_to_write)
        
        print(f"Successfully scraped product: {name_value} ({sku_value})")
        return True
        
    except Exception as e:
        print(f"Error scraping {product_url}: {e}")
        return False

##### CHẶN THÔNG BÁO #####

options = Options()
options.add_argument("--disable-infobars")
options.add_argument("--disable-notifications")
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option('useAutomationExtension', False)
prefs = {
    "credentials_enable_service": False, 
    "profile.password_manager_enabled": False, 
}
options.add_experimental_option("prefs", prefs)

##### MAIN FUNCTION #####

def crawl_links():
    """
    Hàm thực hiện việc cào tất cả các links sản phẩm từ IvyModa
    """
    paths = initialize_directory_structure(base_save_dir, web_url)
    
    driver = webdriver.Chrome(options=options)
    driver.maximize_window()
    time.sleep(5)
    
    driver.get(web_url)
    time.sleep(5)
    
    # Xóa thông báo popup nếu có
    try:  
        close_button = driver.find_element(By.CSS_SELECTOR, "div.popup-content div.action-close")
        actions = ActionChains(driver)
        actions.move_to_element(close_button).click().perform()
        time.sleep(1)
    except:
        print("Không có thông báo nào.")
    
    # Khởi tạo ActionChains
    actions = ActionChains(driver)
    
    # Tìm tất cả các danh mục từ menu
    menu_element = driver.find_element(By.CSS_SELECTOR, "ul.menu")
    categorys = menu_element.find_elements(By.TAG_NAME, "a")
    

    for index, category in enumerate(categorys):
        if index == 30:
        
            category_href = category.get_attribute('href')
            if category_href and ('danh-muc' in category_href or 'lookbook' in category_href):
                # Lấy tên danh mục từ URL
                pattern = r"(?:danh-muc|lookbook)/([^/?#]+)"
                match = re.search(pattern, category_href)
                
                if match:
                    category_name = match.group(1).replace("-", "_")
                else:
                    category_text = category.text.strip()
                    category_name = re.sub(r'[\\/*?:"<>|]', "", category_text.replace(" ", "_").replace("-", "_"))
                
                print(f"Opening: {category_href} (Category: {category_name})")
                
                # Mở tab mới cho danh mục
                driver.execute_script(f"window.open('{category_href}');")
                driver.switch_to.window(driver.window_handles[-1])
                time.sleep(5)
                
                page = 1
                has_next_page = True
                
                while has_next_page:
                    # Lấy thông tin phân trang hiện tại nếu có
                    try:
                        pagination = driver.find_element(By.XPATH, '//*[@id="products_active_ts"]')
                        pagination_text = pagination.text
                    except:
                        pagination_text = str(page)
                    
                    print(f"Processing: {category_name} - Page {pagination_text}")
                    
                    # Cuộn xuống để tải hết trang
                    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                    time.sleep(3)
                    
                    # Tạo thư mục category_{pagination}
                    category_dir, _ = create_category_directory(paths['list_view_dir'], category_name, pagination_text)
                    
                    # Lưu HTML của trang danh sách
                    save_html_source(category_dir, driver)
                    
                    # Tìm tất cả sản phẩm trên trang theo CSS selector của IvyModa
                    products = driver.find_elements(By.CSS_SELECTOR, "div.item-cat-product div.product div.thumb-product a")
                    
                    if not products:
                        # Thử selector khác nếu không tìm thấy sản phẩm
                        products = driver.find_elements(By.CSS_SELECTOR, "div.product a.product-link")
                        
                    if not products:
                        # Thử selector khác nếu vẫn không tìm thấy sản phẩm
                        products = driver.find_elements(By.CSS_SELECTOR, "div.product a")
                    
                    if products:
                        # Lưu thông tin sản phẩm và lấy các links
                        product_links = save_product_data(category_dir, products, actions)
                        
                        # Cập nhật các file CSV
                        update_product_links(paths, product_links)
                    else:
                        print(f"Không tìm thấy sản phẩm nào trong trang {pagination_text}")
                    
                    # Kiểm tra nút phân trang next
                    try:
                        next_page = driver.find_element(By.XPATH, '//li[@class="last-page"]/preceding-sibling::li[1]')
                        actions.move_to_element(next_page).click().perform()
                        page += 1
                        time.sleep(5)  # Đợi trang mới tải xong
                    except Exception as e:
                        print(f"Không tìm thấy nút next page hoặc đã hết trang: {e}")
                        has_next_page = False
                
                # Đóng tab sau khi xử lý xong danh mục
                driver.close()
                driver.switch_to.window(driver.window_handles[0])
    driver.quit()
    return paths

def crawl_details(paths):
    """
    Hàm thực hiện việc cào chi tiết tất cả các sản phẩm từ list_add.csv
    """
    # Đọc danh sách links cần cào chi tiết từ list_add.csv
    list_add_path = paths['list_add_path']
    list_done_path = paths['list_done_path']
    
    links_to_scrape = load_csv(list_add_path)
    links_done = load_csv(list_done_path)
    
    # Lọc ra các links chưa được cào
    links_to_scrape = links_to_scrape - links_done
    
    if not links_to_scrape:
        print("Không có links mới cần cào chi tiết")
        return
    
    print(f"Bắt đầu cào chi tiết cho {len(links_to_scrape)} sản phẩm mới")
    
    # Khởi tạo driver mới
    driver = webdriver.Chrome(options=options)
    driver.maximize_window()
    time.sleep(2)
    
    # Cào chi tiết từng sản phẩm
    for i, link in enumerate(links_to_scrape):
        print(f"Cào chi tiết sản phẩm {i+1}/{len(links_to_scrape)}: {link}")
        success = scrape_product_detail(driver, link, paths)
        
        if success:
            print(f"Hoàn thành cào chi tiết: {link}")
        else:
            print(f"Thất bại khi cào chi tiết: {link}")
        
        # Delay giữa các lần request để tránh bị block
        time.sleep(2)
    
    # Hoàn thành và đóng trình duyệt
    print(f"Hoàn thành cào chi tiết cho {len(links_to_scrape)} sản phẩm")
    driver.quit()


def main():
    """
    Hàm chính thực hiện quá trình cào: 
    1. Cào tất cả links sản phẩm từ categories 
    2. Sau đó cào chi tiết các sản phẩm
    """
    print("Bắt đầu quá trình cào dữ liệu...")
    
    print("=== PHASE 1: Crawling category links ===")
    paths = crawl_links()
    
    print("=== PHASE 2: Crawling product details ===")
    crawl_details(paths)
    
    print("Hoàn thành toàn bộ quá trình cào dữ liệu!")

if __name__ == "__main__":
    main()
  